In [1]:
import pandas as pd
import numpy as np
import random
import re
import csv
from collections import defaultdict
from gensim import corpora, models, similarities, matutils
from string import punctuation
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import nltk
from nltk import RegexpTokenizer, word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')  
%matplotlib inline

In [2]:
Vet_dreams_clusters_kmeans =np.array(pd.read_csv('../data/sa/vet_clusters_kmeans.txt', header=None, names=['Cluster'] )['Cluster'])
print(len(Vet_dreams_clusters_kmeans))
#Vet_dreams_clusters_kmeans= np.array([x+1 for x in Vet_dreams_clusters_kmeans])

593


In [3]:
Phil_dreams_clusters_kmeans=[4, 3, 1, 4, 2, 1, 3, 0, 2, 1, 3, 2, 1, 3, 4, 4, 4, 2, 4, 3, 4, 2,
       4, 1, 2, 1, 4, 1, 4, 4, 1, 3, 4, 1, 4, 2, 1, 2, 2, 1, 2, 1, 0, 4,
       3, 1, 4, 4, 1, 1, 4, 4, 1, 0, 4, 0, 4, 1, 1, 4, 1, 4, 1, 4, 1, 4,
       3, 4, 2, 2, 1, 4, 1, 1, 2, 0, 2, 4, 2, 4, 4, 1, 1, 1, 4, 3, 1, 4,
       1, 1, 1, 2, 2, 1, 0, 2, 1, 4, 1, 2, 2, 4, 1, 1, 1, 1, 4, 0, 0, 0,
       4, 2, 2, 0, 2, 4, 3, 0, 2, 1, 2, 3, 1, 2, 4, 2, 1, 0, 4, 3, 3, 1,
       2, 0, 4, 2, 1, 0, 1, 4, 2, 2, 1, 2, 2, 1, 4, 2, 3, 4, 4, 0, 0, 3,
       4, 2, 1, 0, 4, 4, 4, 0, 4, 1, 4, 4, 2, 1, 1, 2, 0, 2, 2, 1, 1, 0,
       4, 2, 4, 0, 4, 3, 1, 4, 2, 4, 4, 2, 0, 3, 1, 4, 0, 4, 2, 4, 4, 2,
       4, 2, 1, 1, 4, 2, 4, 1, 4, 4, 3, 0, 2, 0, 2, 4, 2, 4, 2, 4, 0, 4,
       0, 3, 4, 4, 1, 3, 2, 2, 2, 1, 0, 2, 2, 1, 2, 2, 1, 3, 4, 2, 1, 0,
       0, 4, 0, 4, 0, 4, 4, 2, 1, 0, 4, 1, 2, 2, 2, 4, 2, 4, 1, 1, 0, 0,
       3, 2, 2, 3, 1, 4, 1, 4, 4, 4, 2, 2, 3, 4, 4, 4, 2, 4, 4, 1, 4, 0,
       2, 1, 2, 1, 2, 0, 4, 1, 1, 0, 2, 2, 0, 4, 4, 2, 1, 2, 4, 2, 2, 1,
       2, 4, 1, 1, 2, 0, 4, 2, 2, 4, 2, 0, 4, 2, 2, 1, 2, 3, 4, 3, 3, 3,
       2, 1, 2, 4, 4, 1, 2, 1, 1, 4, 0, 4, 4, 0, 4, 3, 0, 4, 0, 4, 1, 4,
       4, 1, 2, 4, 1, 2, 4, 4, 3, 4, 4, 4, 2, 1, 1, 0, 1, 0, 4, 2, 1, 4,
       1, 4, 0, 0, 3, 0, 1, 4, 0, 1, 2, 0, 1, 2, 0, 1, 0, 1, 2, 1, 2, 0,
       4, 0, 0, 3, 4, 1, 4, 4, 2, 0, 1, 2, 4, 2, 1, 2, 4, 2, 3, 4, 4, 2,
       1, 1, 4, 0, 2, 4, 4, 0, 4, 2, 4, 0, 2, 1, 4, 0, 2, 2, 2, 1, 1, 4,
       0, 1, 2, 3, 0, 0, 0, 0, 0, 3, 0, 2, 4, 2, 0, 1, 0, 0, 0, 3, 4, 4,
       2, 2, 0, 0, 4, 0, 0, 4, 0, 2, 2, 0, 2, 2, 4, 2, 4, 4, 4, 4, 1, 4,
       2, 4, 2, 4, 1, 1, 4, 3, 4, 4, 0, 2, 2, 2, 0, 4, 0, 4, 4, 0, 4, 3]
#Phil_dreams_clusters_kmeans= np.array([x+1 for x in Phil_dreams_clusters_kmeans])

In [4]:
def func_detecta_recurrencia_max(p_clu, p_dreams_clusters):
    cluster=[]
    for i, x in enumerate(p_dreams_clusters):
        if (x!=p_clu): 
            cluster.append(0)
        else:
            cluster.append(1)
    # window size = 10 
    f = pd.DataFrame({'flag_on': cluster})
    resul=f.rolling(10).sum()
    max_recu=resul["flag_on"].max()
    median_recu=resul["flag_on"][resul["flag_on"]>1].median() 
    print("cluster=",p_clu, " max=", max_recu, " mediana=", median_recu)
    
    return(resul)


In [5]:
Phil_res_recu_clusters=[]
Phil_res_recu_max=[]
Phil_res_recu_median=[]
for i in (list(range(5))):
    Phil_res_recu_clusters.append(func_detecta_recurrencia_max(i, Phil_dreams_clusters_kmeans))
    Phil_res_recu_max.append(Phil_res_recu_clusters[i]["flag_on"].max() ) 
    Phil_res_recu_median.append(Phil_res_recu_clusters[i]["flag_on"][Phil_res_recu_clusters[i]["flag_on"]>1].median() ) 

cluster= 0  max= 6.0  mediana= 2.0
cluster= 1  max= 7.0  mediana= 3.0
cluster= 2  max= 7.0  mediana= 3.0
cluster= 3  max= 4.0  mediana= 2.0
cluster= 4  max= 7.0  mediana= 3.0


In [6]:
Vet_res_recu_clusters=[]
Vet_res_recu_max=[]
Vet_res_recu_median=[]
for i in (list(range(8))):
    Vet_res_recu_clusters.append(func_detecta_recurrencia_max(i, Vet_dreams_clusters_kmeans))
    Vet_res_recu_max.append(Vet_res_recu_clusters[i]["flag_on"].max() ) 
    Vet_res_recu_median.append(Vet_res_recu_clusters[i]["flag_on"][Vet_res_recu_clusters[i]["flag_on"]>1].median() ) 

cluster= 0  max= 4.0  mediana= 2.0
cluster= 1  max= 7.0  mediana= 2.5
cluster= 2  max= 5.0  mediana= 2.0
cluster= 3  max= 5.0  mediana= 2.0
cluster= 4  max= 6.0  mediana= 3.0
cluster= 5  max= 5.0  mediana= 2.0
cluster= 6  max= 4.0  mediana= 2.0
cluster= 7  max= 6.0  mediana= 2.0


In [7]:
Vet_lista_color_recu=[]
for i in  list(range(len(Vet_dreams_clusters_kmeans))):
    if  Vet_res_recu_clusters[Vet_dreams_clusters_kmeans[i]]["flag_on"][i] > Vet_res_recu_median[Vet_dreams_clusters_kmeans[i]] :
        Vet_lista_color_recu.append("blue")
    elif  Vet_res_recu_clusters[Vet_dreams_clusters_kmeans[i]]["flag_on"][i] == round(Vet_res_recu_median[Vet_dreams_clusters_kmeans[i]]) :
        Vet_lista_color_recu.append("cyan")
    else:
        Vet_lista_color_recu.append("gray")
        
    

In [8]:
Phil_lista_color_recu=[]
for i in  list(range(len(Phil_dreams_clusters_kmeans))):
    if  Phil_res_recu_clusters[Phil_dreams_clusters_kmeans[i]]["flag_on"][i] > Phil_res_recu_median[Phil_dreams_clusters_kmeans[i]] :
        Phil_lista_color_recu.append("blue")
    elif  Phil_res_recu_clusters[Phil_dreams_clusters_kmeans[i]]["flag_on"][i] == round(Phil_res_recu_median[Phil_dreams_clusters_kmeans[i]]) :
        Phil_lista_color_recu.append("cyan")
    else:
        Phil_lista_color_recu.append("gray")
        
    

In [9]:


with open('../data/sa/phil_color_recu.txt', 'w') as filehandle:
    for listitem in Phil_lista_color_recu:
        filehandle.write('%s\n' % listitem)

In [10]:

with open('../data/sa/vet_color_recu.txt', 'w') as filehandle:
    for listitem in Vet_lista_color_recu:
        filehandle.write('%s\n' % listitem)